**MACHINE LEARNING**

**HRV for detecting arousal**

In [3]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut, KFold, GridSearchCV
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor


In [4]:
# Loading the dataset
file = r"C:\Users\Salin\OneDrive\Documentos\ESSEX\DSPROJECT\PPG_HR_Analysis_Longer_Intervals\Grey_Cal\norm_Grey_Cal_Start_arousal.csv"
df = pd.read_csv(file)

# Displaying info
df.head()

,Participant,Stimulus,Mean RR,Median RR,SDNN,RMSSD,NN50,pNN50,aVLF,aLF,...,F2_MinArea,F3_MaxArea,F4_MeanArea,F5_StdX,F6_StdY,F7_SkewX,F8_SkewY,F9_KurtX,F10_KurtY,Arousal
0,7yqP3,HN_1,0.039210,0.093410,3.228654,1.120471,0.166667,1.336744,3.825702,3.813462,...,34.652642,-0.376342,0.925245,-0.981104,0.484386,-2.965765,0.550055,-0.512987,1.708991,0.163328
1,7yqP3,HN_2_H,0.166076,0.195852,0.243717,0.027057,-0.833333,-0.631082,0.411365,0.543677,...,10.188483,-0.186720,0.597721,-0.910749,0.001414,-2.237194,0.001238,-0.402162,0.002464,0.242072
2,7yqP3,HN_2_L,0.147075,0.133413,0.512842,0.319530,-1.000000,-1.000000,1.460010,1.646523,...,14.390826,-0.383948,0.740808,-0.933247,-0.026650,-1.942930,-0.031684,-0.594319,-0.076915,-0.302404
3,7yqP3,HN_3_H,-0.048725,0.036448,2.725267,-0.041174,-0.500000,-0.280134,1.228109,0.365673,...,26.289432,-0.162096,0.856847,-0.964765,-0.014276,1.669547,-0.017171,0.026110,-0.042298,0.246764
4,7yqP3,HN_3_L,0.040518,0.068274,0.045487,0.178456,-0.666667,-0.332359,0.726585,0.904300,...,14.476670,-0.229274,0.610427,-0.936812,0.022560,-0.997235,0.027340,-0.344528,0.069222,-0.318291


In [5]:
X = df.drop(columns=['Participant', 'Stimulus', 'Arousal'])
y = df['Arousal']
groups = df['Participant']

In [6]:
def evaluate_regression(true_values, predicted_values):
    mse = mean_squared_error(true_values, predicted_values)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true_values, predicted_values)
    r2 = r2_score(true_values, predicted_values)

    print(f"RMSE: {rmse:.3f}")
    print(f"MAE: {mae:.3f}")
    print(f"R² Score: {r2:.3f}")

    return

**XGBOOST REGRESSOR**

In [7]:
logo = LeaveOneGroupOut()

In [8]:
true_values = []
predicted_values = []

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    true_values.extend(y_test)
    predicted_values.extend(y_pred)

print("LOPO without hyperparameter tuning:")
evaluate_regression(true_values, predicted_values)


LOPO without hyperparameter tuning:
RMSE: 0.285
MAE: 0.233
R² Score: -0.006


In [9]:
global_corr = X.corrwith(y)
global_corr = global_corr.sort_values(key=abs, ascending=False)
global_corr.head(10)

F1_CrossingPoints   -0.199111
NN50                -0.153291
F3_MaxArea          -0.146636
F9_KurtX            -0.079978
F4_MeanArea          0.057464
F2_MinArea           0.051227
F10_KurtY            0.037820
F6_StdY              0.037238
F8_SkewY             0.036542
pNN50               -0.033472
dtype: float64

In [10]:
# Checking number of missing values per column
missing_counts = X.isna().sum()

# Displaying only columns that have missing values
missing_counts[missing_counts > 0].sort_values(ascending=False)

NN50               342
pNN50              342
Parasympathetic     19
F2_MinArea          19
F3_MaxArea          19
F4_MeanArea         19
F5_StdX             19
F6_StdY             19
F7_SkewX            19
F8_SkewY            19
F9_KurtX            19
F10_KurtY           19
dtype: int64

In [11]:
selected_features = ['F1_CrossingPoints', 'NN50']  
X = df[selected_features]

In [12]:
true_values = []
predicted_values = []

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    true_values.extend(y_test)
    predicted_values.extend(y_pred)

print("LOPO without hyperparameter tuning with most significant features:")
evaluate_regression(true_values, predicted_values)

LOPO without hyperparameter tuning with most significant features:
RMSE: 0.316
MAE: 0.255
R² Score: -0.233


In [13]:
true_values = []
predicted_values = []
results = [] 
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    test_participant = groups.iloc[test_idx].unique()[0]

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    grid = GridSearchCV(XGBRegressor(random_state=42), param_grid,
                        cv=inner_cv, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_

    y_pred = best_model.predict(X_test)

    true_values.extend(y_test)
    predicted_values.extend(y_pred)

    rmse = root_mean_squared_error(y_test, y_pred)

    # Saving Parameters 
    results.append({
        'Participant': test_participant,
        'RMSE': rmse,
        **grid.best_params_
    })

print("Nested LOPO with hyperparameter tuning:")
evaluate_regression(true_values, predicted_values)

# For further use if necessary to check the best parameters
results_df = pd.DataFrame(results)
params_df = results_df.drop(columns=['Participant', 'RMSE'])

Nested LOPO with hyperparameter tuning:
RMSE: 0.269
MAE: 0.236
R² Score: 0.103


Best Parameters per Participant

To understand whether certain model configurations generalize better, the best hyperparameters selected during each LOPO iteration were tracked.

Below:
- The most commonly selected configuration.
- The configuration that resulted in the lowest average RMSE across participants.


In [14]:
# The most used configuration
most_common_configs = (
    params_df.value_counts()
    .reset_index(name='Count')
    .sort_values(by='Count', ascending=False)
    .head(3)
)
print("Most Common Configurations:")
display(most_common_configs)

# The best config for better results
best_avg_configs = (
    results_df
    .groupby(['n_estimators', 'max_depth', 'learning_rate', 'subsample'])['RMSE']
    .mean()
    .reset_index()
    .sort_values(by='RMSE')
    .head(3)
)
print("Best Performing Configurations (Lowest Avg. RMSE):")
display(best_avg_configs)


Most Common Configurations:


,learning_rate,max_depth,n_estimators,subsample,Count
0,0.01,3,200,0.8,43


Best Performing Configurations (Lowest Avg. RMSE):


,n_estimators,max_depth,learning_rate,subsample,RMSE
0,200,3,0.01,0.8,0.263863


**Random Forest Regressor**

In [ ]:
X = X.fillna(X.mean())

true_values = []
predicted_values = []

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    true_values.extend(y_test)
    predicted_values.extend(y_pred)

print("R")
evaluate_regression(true_values, predicted_values)
    

RMSE: 0.303
MAE: 0.246
R² Score: -0.137


In [ ]:
# Resultados finales
true_values = []
predicted_values = []
best_params_list = []

# Hyperparameters tunning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

    grid = GridSearchCV(
        estimator=RandomForestRegressor(random_state=42),
        param_grid=param_grid,
        cv=inner_cv,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_

    y_pred = best_model.predict(X_test)

    true_values.extend(y_test)
    predicted_values.extend(y_pred)
    best_params_list.append(grid.best_params_)

evaluate_regression(true_values, predicted_values)

RMSE: 0.281
MAE: 0.234
R² Score: 0.023


In [18]:
best_params_df = pd.DataFrame(best_params_list)
print("\nBest hyperparameters per participant:")
display(best_params_df.value_counts().head())


Best hyperparameters per participant:


max_depth  min_samples_leaf  min_samples_split  n_estimators
10         2                 5                  100             25
                                                200             17
           1                 5                  100              1
Name: count, dtype: int64